# 1、使用FUNCTION_CALL模式

In [10]:
# 0、导入相关的依赖包
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 加载环境变量
dotenv.load_dotenv()

# 1、获取 tavily 的实例
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

search = TavilySearchResults(max_results=1)

# 2、获取对应的工具的实例
search_tool = Tool(
    name='Search',
    func=search.run,
    description='用于检索互联网上的信息'
)

# 3、获取大模型
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 4、创建提示词模板（以ChatPromptTemplate为例）
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', '您是一位乐于助人的助手, 请务必使用 Search 工具来获取信息。'),   # 此处的工具名称需要和上面定义的 search_tool 里面的名称相匹配
        ('human', '{input}'),
        # ('system', '{agent_scratchpad}'),
        ('placeholder', '{agent_scratchpad}'),
    ]
)

# 5、获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt_template
)

# 6、获取AgentExecutor的实例
agent_executor = AgentExecutor(agent=agent, tools=[search_tool], verbose=True)

# 7、通过实例调用invoke()方法, 得到响应
response = agent_executor.invoke({'input': '今天北京的天气怎么样？'})

# 8、处理响应结果
print(response)





> Entering new AgentExecutor chain...

Invoking: `Search` with `北京天气预报`


[{'url': 'https://www.nmc.cn/publish/forecast/ABJ/beijing.html', 'content': '+ 全球天气公报\n  + 全球热带气旋监测公报\n  + WMO第XI海区海事天气公报\n  + 全球灾害性天气监测月报\n  + 全球雨雪落区预报\n  + 世界气象中心（北京）门户网\n  + 一带一路气象服务\n  + 亚洲沙尘暴预报专业气象中心\n 环境气象\n\n  + 雾预报\n  + 霾预报\n  + 沙尘天气预报\n  + 空气污染气象条件预报\n  + 环境气象公报\n  + 大气环境气象公报\n 农业气象\n\n  + 土壤水分监测\n  + 农业干旱综合监测\n  + 关键农时农事\n  + 农业气象周报\n  + 农业气象月报\n  + 生态气象监测评估\n  + 农业气象专报\n  + 作物发育期监测\n  + 农业气象灾害风险预警\n  + 国外农业气象月报\n 数值预报\n\n  + CMA全球天气模式\n  + CMA全球集合模式\n  + CMA区域模式\n  + CMA区域集合模式\n  + CMA台风模式\n  + 海浪模式\n\n1. 当前位置：首页\n2. 北京市\n3. 北京天气预报\n\n省份：城市：\n\n日出\n\n北京\n\n日落\n\n降水量\n\n相对湿度\n\n体感温度\n\n雷达图 [...] 北京 18.3℃ 西南风 微风\n\n地质灾害气象风险预警\n\n中小河流洪水气象风险预警\n\n渍涝风险气象预警\n\n交通气象预报\n\n森林火险预报\n\n草原火险预报\n\n台风海洋\n\n台风快讯与报文\n\n台风路径预报\n\n台风公报\n\n台风预警\n\n海区预报\n\n海事公报\n\n海洋天气预报\n\n近海海雾预报\n\n海区风力预报\n\n北太平洋分析与预报\n\n全球热带气旋监测公报\n\n台风命名\n\n台风综合信息\n\n全球预报\n\n全球天气公报\n\n全球热带气旋监测公报\n\nWMO第XI海区海事天气公报\n\n全球灾害性天气监测月报\n\n全球雨雪落区预报\n\n世界气象中心（

说明：创建提示词模板中的agent_scratchpad必须声明, 它用于存储和传递Agent的思考过程。 比如, 在调用链式工具时（如先搜索天气再推荐行程）, agent_scratchpad保留所有历史步骤, 避
免上下文丢失。format方法会将intermediate_steps转换为特定格式的字符串, 并赋值给agent_scratchpad变量。如果不传递intermediate_steps参数, 会导致KeyError：'intermediate_steps' 错误。



# 2、ReAct模式

## 2.1 使用PromptTemplate


举例1：

In [8]:

# 1、导入相关的包
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent

# 加载环境变量
dotenv.load_dotenv()

# 2、定义搜索化工具
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

search = TavilySearchResults(max_results=1)

tools = [search]

# 3、自定义提示词模板
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)

# 4、定义LLM
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 5、创建Agent对象
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# 6、创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7、测试
response = agent_executor.invoke({'input': '今天北京的天气怎么样？'})
print(response)



> Entering new AgentExecutor chain...
Action: tavily_search_results_json  
Action Input: "今天 北京 天气"  [{'url': 'https://www.weather.com.cn/weather/101010100.shtml', 'content': '首页 预报 预警 雷达 云图 天气地图 专业产品 资讯 视频 节气 我的天空\n\n更多\n\n台风路径 空间天气 图片 专题 环境 旅游 碳中和 气象科普 一带一路 产创平台\n\n国内) 本地) 国际)\n\n:   北京 上海 成都 杭州 南京 天津 深圳 重庆 西安 广州 青岛 武汉\n\n:   故宫 阳朔漓江 龙门石窟 野三坡 颐和园 九寨沟 东方明珠 凤凰古城 秦始皇陵 桃花源\n\n高球\n:   佘山 春城湖畔 华彬庄园 观澜湖 依必朗 旭宝 博鳌 玉龙雪山 番禺南沙 东方明珠\n\n<<返回 全国\n\n河北下辖区域\n\n热门城市\n\n:   曼谷 东京 首尔 吉隆坡 新加坡 巴黎 罗马 伦敦 雅典 柏林 纽约 温哥华 墨西哥城 哈瓦那 圣何塞 巴西利亚 布宜诺斯艾利斯 圣地亚哥 利马 基多 悉尼 墨尔本 惠灵顿 奥克兰 苏瓦 开罗 内罗毕 开普敦 维多利亚 拉巴特\n\n:   亚洲 欧洲 北美洲 南美洲 非洲 大洋洲\n\n全国> 北京 > 城区\n\n 今天\n 7天\n 8-15天\n 40天\n 雷达图\n\n # 13日（今天）\n\n  晴\n\n  14/1℃\n\n  <3级\n # 14日（明天）\n\n  晴 [...] 无需担心过敏，可放心外出，享受生活。\n 较冷 穿衣指数\n\n  建议着厚外套加毛衣等服装。)\n 适宜 洗车指数\n\n  天气较好，适合擦洗汽车。\n 中等 紫外线指数\n\n  涂擦SPF大于15、PA+防晒护肤品。\n\n# 天气资讯\n\n:   北京今明天晴朗天气在线 昼夜温差较大最低气温1℃左右  中国天气网 2025-11-13 07:01\n\n:   台风蓝色预警：“凤凰”今天将变性为温带气旋 东海等海域有大风  中国天气网 2025-11-13 06:05\n\n:   台风“凤凰”在台湾屏东县沿海登陆 或为今年登陆我国的末台风 

举例2：直接调用官方模板

In [11]:
from langchain import hub
# 1、导入相关的包
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent

# 加载环境变量
dotenv.load_dotenv()

# 2、定义搜索化工具
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

search = TavilySearchResults(max_results=1)

tools = [search]

# 3、自定义提示词模板
# template = """
# Answer the following questions as best you can. You have access to the following tools:
# 
# {tools}
# 
# Use the following format:
# 
# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question
# 
# Begin!
# 
# Question: {input}
# Thought:{agent_scratchpad}
# """

# prompt = PromptTemplate.from_template(template)
# 使用LangChain Hub中的官方ReAct提示模板
prompt = hub.pull('hwchase17/react')

# 4、定义LLM
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 5、创建Agent对象
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# 6、创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7、测试
response = agent_executor.invoke({'input': '今天北京的天气怎么样？'})
print(response)

D:\develop\conda_envs\langchain310\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
我需要查找今天北京的天气情况。  
Action: tavily_search_results_json  
Action Input: "今天 北京 天气"  [{'url': 'https://zh.climate-data.org/%E4%BA%9E%E6%B4%B2/%E4%B8%AD%E5%9B%BD/%E5%8C%97%E4%BA%AC%E5%B8%82/%E5%8C%97%E4%BA%AC%E5%B8%82-134/t/%E5%8D%81%E4%B8%80%E6%9C%88-11/', 'content': '在 十一月 期間，北京市 的每日降水量介於 0.5 毫米／英吋與 6.3 毫米／英吋之間。\n11.11 是最潮濕的一天，降水量達 6.3 毫米／英吋，而 01.11 則是最乾燥的一天，降水量僅有 0.5 毫米／英吋。\n月初期間，平均降雨量為 1.8 毫米／英吋，降雨日數為 2 天。\n中旬期間，平均降雨量達到 2.3 毫米／英吋，降雨日數為 2 天。\n月末期間，平均降雨量為 2.3 毫米／英吋，降雨日數為 2 天。\n降雨最多的連續 10 天期間是 11-20，平均降雨量為 2.3 毫米／英吋，最乾燥的連續 10 天則為 1-10，平均降雨量為 0.5 毫米／英吋。\n\n在 北京市 ，十一月 期間的濕度範圍從 67% 到 74%。最高濕度出現在 15.11 ，為 74%，最低濕度則是在 03.11 ，為 67%。濕度在月初平均為 70%，月中為 72%，月底則為 71%。最潮濕的十天期間為 11-20，而最乾燥的十天則落在 1-10。\n\n## 北京市 各月份天氣 // 天氣平均值\n\n<> [...] 北京市 在 十一月 中最暖和的一天是 03.11，達到 15.5°C，而 30.11 則是最寒冷的一天，最低溫為 -4.5°C。 前十天的最高溫平均為 13.8°C，最低溫約為 1.6°C。 從第十一天到第二十天，預期最高溫約為 9.8°C，最低溫約為 -1.6°C。 最後這十天期間，最高溫平均為 6.8°C，最低溫則約為 -3.8°C。 最暖十大天段落位於 1-10，最高溫平均為 7.4°C。 最寒冷的期間是 21-30，最低溫約為 0.8°C。\n\n


## 2.2 使用ChatPromptTemplate

In [ ]:

# 1、导入相关的包
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent

# 加载环境变量
dotenv.load_dotenv()

# 2、定义搜索化工具
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

search = TavilySearchResults(max_results=1)

tools = [search]

# 3、自定义提示词模板
template = """
Answer the following questions as best you can. You have access to the following tools: {tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}
"""

prompt = ChatPromptTemplate.from_template(template)

# 4、定义LLM
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 5、创建Agent对象
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# 6、创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7、测试
response = agent_executor.invoke({'input': '今天北京的天气怎么样？'})
print(response)